In [3]:
from sklearn import preprocessing
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

## Preprocessing Data with Min-Max Scaling

In [5]:
df = pd.read_csv("master.csv")
df['GDP'] = df['gdp_for_year ($)'].str.replace(",", "")
df['GDP'] = df['GDP'].astype('int64')
df = df.drop("HDI for year", 1)
df = df.drop("gdp_for_year ($)", 1)
df['GDP_per_capita'] = df['gdp_per_capita ($)']
df = df.drop("gdp_per_capita ($)", 1)
data_df = df[['year','sex','age','GDP', 'GDP_per_capita', 'generation']]
min_max_scaler = preprocessing.MinMaxScaler()
X = data_df
y = df['suicides/100k pop']
X_minmax = min_max_scaler.fit_transform(X)

In [6]:
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X_minmax)
poly_model = LinearRegression()
parameters = {'fit_intercept': [True, False], 'normalize': [True, False], 'n_jobs': [0,1,2,5]}
gsp_model = GridSearchCV(poly_model, parameters,scoring='r2')
gsp_model.fit(X_poly, y)
print("Best combination: ", gsp_model.best_params_)
print("Performance: ", gsp_model.best_score_)
poly_model = LinearRegression(fit_intercept= True, n_jobs= 0, normalize= True)
poly_model.fit(X_poly, y)
CVP = cross_val_score(poly_model, X_poly, y, cv=3,scoring='r2')
print("CVP: ", CVP.mean())

Best combination:  {'fit_intercept': True, 'n_jobs': 0, 'normalize': True}
Performance:  0.23033694167025268
CVP:  0.2969969270021437


In [7]:
SGD_model = SGDRegressor()
parameters = {'loss': [ 'huber', 'epsilon_insensitive'], 'penalty': ['l2', 'l1', 'elasticnet'],'fit_intercept': [True, False]}
gss_model = GridSearchCV(SGD_model, parameters,scoring='r2')
gss_model.fit(X_poly, y)
print("Best combination: ", gss_model.best_params_)
print("Performance: ", gss_model.best_score_)

SGD_model = SGDRegressor(fit_intercept= True, loss= 'epsilon_insensitive', penalty= 'l1')
SGD_model.fit(X_poly, y)
CVS = cross_val_score(SGD_model, X_poly, y, cv=3,scoring='r2')
print("CVS: ", CVS.mean())

Best combination:  {'fit_intercept': False, 'loss': 'epsilon_insensitive', 'penalty': 'l1'}
Performance:  0.27508510120007684
CVS:  0.2719772118330209
